<a href="AIS3_Lab7_threat_report_embedding_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

In [5]:
import pandas as pd

# Load the uploaded file
file_path = 'Datasets/ThreatActor/ThreatActor_cleaned.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)
df

,source name,mapping description
0,APT-C-36,APT-C-36 has downloaded binary data from a spe...
1,APT-C-36,APT-C-36 has prompted victims to accept macros...
2,APT-C-36,APT-C-36 has disguised its scheduled tasks as ...
3,APT-C-36,APT-C-36 has used port 4050 for C2 communicati...
4,APT-C-36,APT-C-36 has used ConfuserEx to obfuscate its ...
...,...,...
3382,menuPass,menuPass has used and modified open-source too...
3383,menuPass,menuPass has used legitimate access granted to...
3384,menuPass,menuPass has used valid accounts including sha...
3385,menuPass,menuPass executes commands using a command-lin...


In [6]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.head()

,source name,mapping description
0,APT3,APT3 has a tool that can delete files.
1,Equation,Equation has used an encrypted virtual file sy...
2,OilRig,OilRig has used LinkedIn to send spearphishing...
3,APT18,APT18 uses cmd.exe to execute commands on the ...
4,Mofang,Mofang has compressed the ShimRat executable w...


In [7]:
len(set(df["source name"].to_list()))

143

In [8]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['mapping description'])

# Labels
y = df['source name']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Use RandomForestClassifier for classification
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the classification performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)


print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)


/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9867256637168141,
 '                    precision    recall  f1-score   support\n\n          APT-C-36       1.00      1.00      1.00         2\n              APT1       1.00      1.00      1.00         4\n             APT12       1.00      0.33      0.50         3\n             APT17       1.00      1.00      1.00         1\n             APT18       1.00      1.00      1.00         2\n             APT19       1.00      1.00      1.00         1\n             APT28       1.00      1.00      1.00        17\n             APT29       1.00      1.00      1.00        13\n              APT3       1.00      1.00      1.00         7\n             APT32       1.00      1.00      1.00         7\n             APT33       1.00      1.00      1.00         4\n             APT37       1.00      1.00      1.00         8\n             APT38       1.00      1.00      1.00         4\n             APT39       1.00      1.00      1.00        12\n             APT41       0.93      1.00      0.97        14\

In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the text descriptions using SentenceBERT
sentence_embeddings = model.encode(df['mapping description'],device=device)

# Labels
y = df['source name']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, y, test_size=0.2, shuffle=False)

# Use RandomForestClassifier for classification
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the classification performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)


/opt/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Accuracy: 0.7890855457227138
Classification Report:
                    precision    recall  f1-score   support

          APT-C-36       1.00      0.50      0.67         2
              APT1       1.00      0.25      0.40         4
             APT12       0.00      0.00      0.00         3
             APT17       1.00      1.00      1.00         1
             APT18       1.00      0.50      0.67         2
             APT19       0.00      0.00      0.00         1
             APT28       0.77      1.00      0.87        17
             APT29       0.62      0.77      0.69        13
              APT3       0.71      0.71      0.71         7
             APT32       0.39      1.00      0.56         7
             APT33       0.75      0.75      0.75         4
             APT37       1.00      0.25      0.40         8
             APT38       0.33      0.25      0.29         4
             APT39       0.79      0.92      0.85        12
             APT41       0.64      1.00      0.

/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packag

In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np



model = SentenceTransformer('all-MiniLM-L6-v2')

sentence_embeddings = model.encode(df['mapping description'])

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['source name'])

X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, y_encoded, test_size=0.2, shuffle=False)

train_embeddings = {label: [] for label in np.unique(y_train)}
for emb, label in zip(X_train, y_train):
    train_embeddings[label].append(emb)

y_pred = []
for test_emb in X_test:
    similarities = []
    for label in np.unique(y_train):
        avg_train_emb = np.mean(train_embeddings[label], axis=0)
        similarity = cosine_similarity([test_emb], [avg_train_emb])[0][0]
        similarities.append((label, similarity))
    best_label = max(similarities, key=lambda x: x[1])[0]
    y_pred.append(best_label)

y_test = label_encoder.inverse_transform(y_test)
y_pred = label_encoder.inverse_transform(y_pred)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_rep)


Accuracy: 0.9660766961651918
Classification Report:
                    precision    recall  f1-score   support

          APT-C-36       1.00      1.00      1.00         2
              APT1       1.00      0.75      0.86         4
             APT12       0.00      0.00      0.00         3
             APT17       1.00      1.00      1.00         1
             APT18       1.00      1.00      1.00         2
             APT19       1.00      1.00      1.00         1
             APT28       0.94      0.94      0.94        17
             APT29       1.00      1.00      1.00        13
              APT3       1.00      1.00      1.00         7
             APT30       0.00      0.00      0.00         0
             APT32       0.70      1.00      0.82         7
             APT33       1.00      1.00      1.00         4
             APT37       1.00      0.88      0.93         8
             APT38       1.00      1.00      1.00         4
             APT39       1.00      0.92      0.

/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packag